# Simple Bigram Model

### A simple bigram model will help motivate more intricate transformer architectures.

## What is a Bigram?

From Wikipedia:

A bigram or digram is a sequence of two adjacent elements from a string of tokens, which are typically letters, syllables, or words. A bigram is an n-gram for n=2.

The frequency distribution of every bigram in a string is commonly used for simple statistical analysis of text in many applications, including in computational linguistics, cryptography, and speech recognition.

## Bigram Model

A bigram model then involves predicting the following token when given a singular preceeding token. 

Let's spell this out in code.

First import PyTorch in python which is a machine learning framework based on the Torch library and the Python language. It is primarily used for creating deep neural networks.

In [3]:
%pip install torch



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


## Hyperparameters
Now we must define some hyperparameters for our Bigram Model. 

***These parameters are taken from Andrej Karpathy's Let's Build GPT YouTube video***

In [5]:
batch_size = 32
block_size = 8 
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

### Batch Size
Batch size is the number of samples used in one forward and backward pass through the network. In principle, batch size determines the number of independent sequences we process in parallel. 

### Block Size
Block size is snynonymous with context length or the context window. It determines the number of tokens considered when predicting a new token.

With our block size of 8, a maximum of 8 tokens will be used to predict the 9th token in the sequence. 

## Seeding the torch.Generator Object

We will be generating random numbers here. To ensure the random numbers are the same everytime we run the preceeding and following code sequence, we must create a torch generator object with a manual seed as follows.

In [6]:
torch.manual_seed(123)

## Read in training corpus

In [20]:
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
# get unique characters
chars = sorted(list(set(text)))
vocab_len = len(chars)
# encode chars as integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

#anon functions to encode from char to integer and decode from integer to char
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

### Split Data into training and validation splits

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

### Function to generate batch of inouts and targets

In [10]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ints = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ints])
    y = torch.stack([data[i+1:i+block_size+1] for i in ints])
    x, y = x.to(device), y.to(device)
    return x, y

### Function to Estimate the Loss

In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## Define Bigram Model Class

In [13]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


### Create Instance of Bigram Model

In [15]:
model = BigramLanguageModel(vocab_len)
m = model.to(device)

### Optimize Estimated Loss with Adam optimizer

Documentation for adam optimizer here: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html

Adam focuses on two ideas

Mometum and RMSprop


Momentum: Helps the optimizer to keep moving in the current direction, similar to the physical concept of momentum. It introduces a moving average of the gradients, and this moving average is then used to update the parameters of the model.

Here's a basic idea of how momentum works:

1. **Update Rule:** Instead of updating the parameters based solely on the current gradient, momentum introduces a moving average of the past gradients. The update rule for a parameter \(w\) becomes:

   $ v_t = \beta \cdot v_{t-1} + (1 - \beta) \cdot \nabla J(w_t) $ \
   $ w_{t+1} = w_t - \alpha \cdot v_t $

   Where:
   - $ \alpha $ is the learning rate.
   - $ \beta$ is the momentum term (typically close to 1, e.g., 0.9 or 0.99).
   - $ \nabla J(w_t)$ is the gradient of the loss with respect to the parameters at the current iteration. 

2. **Benefits:** Momentum helps to smooth out oscillations and speed up convergence, especially in the presence of noisy gradients or if the optimization surface has long, shallow valleys. It helps the optimizer to continue moving in the current direction, even if the gradient changes direction frequently.

3. **Intuition:** Think of it as a ball rolling down a surface with valleys and hills. Momentum allows the ball to accumulate speed when rolling down a slope and carry that speed to overcome small bumps, helping it to converge faster.

RMSprop:

1. **Adaptive Learning Rates:** Similar to Adam, RMSprop adapts the learning rates for each parameter individually. It does so by dividing the learning rate for a parameter by the square root of the exponential moving average of the squared gradients.

   $ v_t = \beta \cdot v_{t-1} + (1 - \beta) \cdot (\nabla J(w_t))^2 $\
   $ w_{t+1} = w_t - \frac{\alpha}{\sqrt{v_t + \epsilon}} \cdot \nabla J(w_t) $

   Where:
   - $ \alpha $ is the learning rate.
   - $ \beta $ is a decay term (typically close to 1, e.g., 0.9).
   - $ \epsilon $ is a small constant added for numerical stability.
   - $ \nabla J(w_t) $ is the gradient of the loss with respect to the parameters at the current iteration. 

2. **Benefits:** RMSprop helps address the problem of vanishing or exploding gradients by normalizing the updates. It is particularly effective in scenarios where the scale of the gradients varies widely across different parameters or time steps.

3. **Exponential Moving Average:** The use of the exponential moving average for squared gradients helps RMSprop to adapt its learning rates dynamically. It focuses more on recent information and less on historical gradients.

4. **Intuition:** RMSprop can be thought of as adjusting the learning rates based on the historical magnitudes of the gradients. It scales down the learning rates for parameters with large and frequent updates, allowing for more stable and efficient convergence.

RMSprop is a key component in the family of adaptive learning rate optimization algorithms and is widely used in practice for training deep neural networks.

In [5]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

NameError: name 'model' is not defined

## Generate From the Model

In [19]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

NameError: name 'm' is not defined

# GPT - Generative Pretrained Transformer

# Understanding the Transformer Architecture

## Motivation/Intuition

The Transformer architecture, introduced in the paper "Attention is All You Need," revolutionized sequence-to-sequence tasks in machine learning. Its key motivation is to capture long-range dependencies in sequences more efficiently, enabling parallelization.

## Components of the Transformer
![Transformer Architecture](https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png)

### 1. Self-Attention Mechanism
![Self-Attention Mechanism](https://theaisummer.com/static/e497f0d469418119f9db9c53b9851e61/b9460/self-attention-explained.png)
#### Overview

Self-attention is a mechanism that allows a model to focus on different parts of input sequences with varying degrees of emphasis. It is particularly popular in natural language processing tasks, such as machine translation and text summarization. The self-attention mechanism enables a model to weigh the importance of different words in a sequence when making predictions, capturing long-range dependencies effectively.

#### Input Representation

Suppose we have an input sequence $ X = \{x_1, x_2, ..., x_n\} $, where $n$ is the sequence length. Each $x_i$ represents the embedding of the $i$-th element in the sequence.

#### Key, Query, and Value Representations

For self-attention, we create three linear projections for each element in the sequence:

- **Key ($K$):** $K = X \cdot W_K$, where $W_K$ is a learnable weight matrix.
- **Query ($Q$):** $Q = X \cdot W_Q$, where $W_Q$ is a learnable weight matrix.
- **Value ($V$):** $V = X \cdot W_V$, where $W_V$ is a learnable weight matrix.

#### Attention Scores

The attention scores are computed by taking the dot product of the query and key representations and then scaling by the square root of the dimension of the key vectors:

$ \text{Attention Scores} = \frac{Q \cdot K^T}{\sqrt{d_k}}$

Here, $d_k$ is the dimension of the key vectors. The scaling helps prevent the dot products from becoming too large, which can lead to overly small gradients in the training process.

#### Softmax and Weighted Sum

$ \text{Softmax}(\mathbf{x})_i = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}} $

Here, $\mathbf{x} = [x_1, x_2, \ldots, x_n]$ is a vector of real numbers, and the softmax function computes the probability distribution over these values. The numerator $e^{x_i}$ represents the exponential of the $i$-th element of the input vector, and the denominator $\sum_{j=1}^{n} e^{x_j}$ is the sum of exponentials over all elements in the vector. The result is a probability distribution where each element is in the range (0, 1), and the sum of all elements is equal to 1.

Apply the softmax function to the attention scores to obtain normalized weights:

$ \text{Attention Weights} = \text{softmax}(\text{Attention Scores}) $


Finally, compute the weighted sum of the value vectors using the attention weights:

$\text{Output} = \text{Attention Weights} \cdot V$

### 2. Multi-Head Attention

![Multi-Head Attention](https://production-media.paperswithcode.com/methods/3080f470-be1b-48d6-b1e8-43edb0a71739.png)

#### Single-Head Attention

For a single head, given a set of input vectors $X = \{x_1, x_2, ..., x_n\}$, and the corresponding query (Q), key (K), and value (V) matrices $W_Q$, $W_K$, and $W_V$, the attention scores for the $i$-th element are computed as follows:

$ \text{Attention}(Q, K, V)_i = \text{softmax}\left(\frac{Q_iK_i^T}{\sqrt{d_k}}\right)V_i $

where $Q_i$, $K_i$, and $V_i$ represent the $i$-th rows of the matrices $Q$, $K$, and $V$, and $d_k$ is the dimension of the key vectors.

#### Multi-Head Attention

For multi-head attention with $H$ heads, let $Q_i^h$, $K_i^h$, and $V_i^h$ denote the query, key, and value vectors for the $i$-th element in the $h$-th head, respectively.

The output $O_i$ for the $i$-th element in the $h$-th head is given by:

$ O_i^h = \text{Attention}(Q^hW_{Qi}, K^hW_{Ki}, V^hW_{Vi}) $

where $W_{Qi}$, $W_{Ki}$, and $W_{Vi}$ are the weight matrices for the $h$-th head.

The final output $O_i$ is obtained by concatenating the outputs from all heads and applying a linear transformation:

$ O_i = \text{Concat}(O_i^1, O_i^2, ..., O_i^H)W_O $

where $W_O$ is the weight matrix for the final linear transformation.


### 3. Positional Encoding

![Positional Encoding](https://machinelearningmastery.com/wp-content/uploads/2022/01/PE1.png)

Positional Encoding Formulas:
1. $ \text{PE}(pos, 2i) = \sin\left(\frac{{pos}}{{10000^{(2i/d)}}}\right) $
2. $ \text{PE}(pos, 2i + 1) = \cos\left(\frac{{pos}}{{10000^{(2i/d)}}}\right) $

Where:
- $ pos $ is the position of the token in the sequence.
- $ i $ is the dimension index of the positional encoding.
- $ d $ is the dimensionality of the positional encoding, typically the same as the embedding dimension.

The positional encoding matrix is then added element-wise to the input embeddings to incorporate positional information in the GPT architecture.


### 4. Encoder and Decoder Stacks

The Transformer architecture comprises stacks of encoders and decoders. The encoder processes the input sequence, while the decoder generates the output sequence. Each encoder and decoder consist of multiple layers, each containing self-attention and feedforward sub-layers.

![Encoder and Decoder Stacks](https://www.researchgate.net/publication/360353665/figure/fig4/AS:1151883645329415@1651641870298/Stacked-encoder-and-decoder-blocks-used-in-Transformers-are-presented-for-the-machine.ppm)

### 5. Feedforward Networks

After the self-attention mechanism, each sub-layer in the encoder and decoder stacks contains a feedforward neural network. This network is responsible for transforming the high-dimensional output of the attention mechanism into a more suitable form for the next layer.

![Feedforward Networks](https://learnopencv.com/wp-content/uploads/2017/10/mlp-diagram.jpg)


In [21]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [14]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [15]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [16]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [17]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_len, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_len)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [24]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # evaluate the loss on train and val sets on varying iterations
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


10.810461 M parameters
step 0: train loss 4.5722, val loss 4.5314
step 500: train loss 1.4393, val loss 2.6423
step 1000: train loss 1.1005, val loss 2.5024


: 

In [1]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

NameError: name 'torch' is not defined